In [50]:
import sys
reload(sys)
sys.setdefaultencoding("utf-8")
sys.path.append('../lib/')
import ImportNotebook
%matplotlib inline

import shelve
import os.path
import numpy as np
import numpy.matrixlib
import scipy.misc
import cv2
import asmgazetracker
import matplotlib.pyplot as plt
import DaugmanIrisLocator as pcl

In [51]:
userProfilePath = '/Users/Jiangzp/trackingdata/user_profile/jzp-68points/trackermodel.yaml'
cameraProfilePath = '/Users/Jiangzp/trackingdata/camera_calibration/iSight/CM_iSight.yaml'
pt = asmgazetracker.ASM_Gaze_Tracker()
pt.initialize(userProfilePath,cameraProfilePath)
pt.isImageRectificationUsingHomoTransformation = 1
pt.poseEstimationUsingRansac = 0
pt.poseEstimationRobustTracker = 0

pt.addReferenceFeature('noseOutterLeft')
pt.addReferenceFeature('noseOutterRight')
# pt.addReferenceFeature('noseInnerLeft')
# pt.addReferenceFeature('noseInnterRight')
pt.addReferenceFeature('noseUnderMiddle')
pt.addReferenceFeature('mouthUpperMiddle')
# pt.addReferenceFeature('mouthUpperInnerLeft')
# pt.addReferenceFeature('mouthUpperInnerRight')
pt.addReferenceFeature('mouthUpperMiddleLeft')
pt.addReferenceFeature('mouthUpperMiddleRight')
pt.changeReferenceFeaturesSet()

In [52]:
# give the full video file path
videoPath = '/Users/Jiangzp/trackingdata/video_parsing/jiangzp/150508_1421vision.mov'

In [53]:
# this value controls how big a image tile is. 
#The tile width = |leftOutterCanthus-rightOutterCanthus|/gridTileNumber
gridTileNumber = 4
folderPath,videoFileName = os.path.split(videoPath)
videoFileRootName,_ = os.path.splitext(videoFileName)
rectifiedImgsPath = os.path.join(folderPath,videoFileRootName)
tileImgsPath = os.path.join(rectifiedImgsPath,str(gridTileNumber))
# save all temp images right besides the video file.
if not os.path.exists(rectifiedImgsPath):
    os.makedirs(rectifiedImgsPath)
if not os.path.exists(tileImgsPath):
    os.makedirs(tileImgsPath)

# check if a pre-calculated dlib parsed result exists.
if os.path.exists(os.path.join(rectifiedImgsPath,'DlibPointsLog')):
    loader2 = shelve.open(os.path.join(rectifiedImgsPath,'DlibPointsLog'))
    DlibPointsLog = loader2['DlibPointsLog']
    loader2.close()
else:
    DlibPointsLog = {}


cap = cv2.VideoCapture(videoPath) 
frameCount = 0
while(True):
    frameCount +=1
    if os.path.exists(os.path.join(rectifiedImgsPath,'r'+str(frameCount)+'.jpg')):
        ret, frame = cap.read() # is this action so buggy....
        rectifiedImage = cv2.imread( os.path.join(rectifiedImgsPath,'r'+str(frameCount)+'.jpg'))
    else:
        ret, frame = cap.read()
        if ret == False:
            break
        frame = cv2.transpose(frame)
        frame = cv2.flip(frame,1)
        frame = frame[350:1080,70:-70,:]
    
    rectifiedDLibPoints = []
    if frameCount in DlibPointsLog:
        rectifiedDLibPoints = DlibPointsLog[frameCount]
    else:
        dlibsuccecced = pt.featureTrackingViaDlib(frame)
        succecced = pt.featureTracking(frame)
        if dlibsuccecced and succecced:
            pt.extractEyeRectifiedImages()
            rectifiedImage = pt.rectifiedImage
#             cv2.imwrite(os.path.join(rectifiedImgsPath,'r'+str(frameCount)+'.jpg'),rectifiedImage)
            rectifiedDLibPoints = pt.allRectifieddLibTrackedPoints2D
            DlibPointsLog[frameCount] = rectifiedDLibPoints
        
        
    if rectifiedDLibPoints:        
        
        rightOutCanthus = rectifiedDLibPoints[36]
        leftOutCanthus = rectifiedDLibPoints[45]
        bottomLine = rectifiedDLibPoints[48][1]
        leftRightLength = np.linalg.norm(rightOutCanthus - leftOutCanthus)
        topLine = int((rightOutCanthus[1]/2 + leftOutCanthus[1]/2).item() - leftRightLength/3*2)
        gridSize = int(leftRightLength/gridTileNumber)
        verticalGridNumber = int(np.round((bottomLine-topLine)/gridSize))
        horizontalGridNumber = int(np.round((int(rightOutCanthus[0].item()) - int(leftOutCanthus[0].item()))/gridSize))+2
        topStart = topLine
        leftStart = int(leftOutCanthus[0].item())-gridSize
        
        tileCount = 0
        for ht in range(horizontalGridNumber):
            for vt in range(verticalGridNumber):
                tileCount += 1
                tile = rectifiedImage[topStart+vt*gridSize:topStart+(vt+1)*gridSize,leftStart+ht*gridSize:leftStart+(ht+1)*gridSize,:]
                tileName = '%03d_%02d_%02d.jpg' % (frameCount,ht,vt)
                cv2.imwrite(os.path.join(tileImgsPath,tileName),tile)
                
        for xpos in range(horizontalGridNumber+1):
            cv2.line(rectifiedImage,(leftStart+xpos*gridSize,topLine),(leftStart+xpos*gridSize,topLine+(verticalGridNumber+1)*gridSize),(0,255,0),2)
        for ypos in range(verticalGridNumber+2):
            cv2.line(rectifiedImage,(leftStart,topStart+ypos*gridSize),(leftStart+(horizontalGridNumber)*gridSize,topStart+ypos*gridSize),(0,255,0),2)
        gridImage = rectifiedImage[topStart:topStart+(verticalGridNumber)*gridSize,leftStart:leftStart+horizontalGridNumber*gridSize,:]
        
        for p in pt.allRectifieddLibTrackedPoints2D:
            cv2.circle(rectifiedImage,tuple(p),3,(0,255,255),-1)
        if pt.allRectifieddLibTrackedPoints2D:
            for p in pt.allConnectionsBetweenDlibTrackedPoints2D:
                cv2.line(rectifiedImage,tuple(pt.allRectifieddLibTrackedPoints2D[p[0].item()]),tuple(pt.allRectifieddLibTrackedPoints2D[p[1].item()]),(0,255,255))
        for p in pt.alldLibTrackedPoints2D:
            cv2.circle(frame,tuple(p),3,(0,255,255),-1)
        if pt.alldLibTrackedPoints2D:
            for p in pt.allConnectionsBetweenDlibTrackedPoints2D:
                cv2.line(frame,tuple(pt.alldLibTrackedPoints2D[p[0].item()]),tuple(pt.alldLibTrackedPoints2D[p[1].item()]),(0,255,255))

        cv2.imwrite(os.path.join(rectifiedImgsPath,'g'+str(frameCount)+'.jpg'),gridImage)
        cv2.imwrite(os.path.join(rectifiedImgsPath,'r'+str(frameCount)+'.jpg'),rectifiedImage)
        cv2.imshow('rectify',rectifiedImage)
        
    cv2.imwrite(os.path.join(rectifiedImgsPath,'u'+str(frameCount)+'.jpg'),frame)
    cv2.imshow('frame',frame)
    
    keyValue = cv2.waitKey(1) & 0xFF 
    if keyValue == ord('q'):
        del(cap)
        cv2.destroyAllWindows()
        break
    elif keyValue == ord('d'):
        pt.reDetectFace()
        
persistence = shelve.open(os.path.join(rectifiedImgsPath,'DlibPointsLog'))
persistence['DlibPointsLog']=DlibPointsLog
persistence.close()

In [54]:
gridImage.shape

(294, 294, 3)